# Street2Sat Point Analysis

<a target="_blank" href="https://colab.research.google.com/github/nasaharvest/street2sat/blob/main/notebooks/deployment_3_analysis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Author:** Ivan Zvonkov

**Last Edited:** Feb 22, 2024

**Description:**
The street2sat pipeline involves transforming GoPro photos of crops into crop type field coordinates. This is done by:
1. **Predicting the crops** visible in the GoPro photo
2. **Road point extraction** Extracting the GPS coordinate of the GoPro photo (will be located on a road)
3. **Obtaining high res imagery** for the GPS coordinate from Mapbox and Bing.
4.  **Drawing an arrow** on the high res satellite imagery from the road point in the direction where the GoPro is pointed.
5. **Field prediction**: Using the high res satellite image with an arrow to predict a mask for the location of the field.
6. **Field point prediction**: Taking the centroid of the predicted field to create a crop type field coordinate.

All of these steps are prone to different failure modes during the development of this pipeline.

The purpose of this notebook is to explore and document these failure modes by analyzing several general points.

## 1. Setup for visualizing all points in Google Earth

1. Download the following files from Github:
- https://github.com/nasaharvest/street2sat/blob/main/data/points/field_bing.geojson
- https://github.com/nasaharvest/street2sat/blob/main/data/points/field_mapbox.geojson
- https://github.com/nasaharvest/street2sat/blob/main/data/points/road.geojson

2. Drag them into Google Earth to visualize.
3. Adjust the colors for easier distinction.

<img src="https://storage.googleapis.com/harvest-public-assets/street2sat/GoogleEarth-updating-colors.png" />


## 2. Setup for individual point analysis

Modify the first cell below to add your own MAPBOX_TOKEN, run the next cells without modification.

In [ ]:
# Set MAPBOX token, for visualizing points on map
MAPBOX_TOKEN = ""
assert MAPBOX_TOKEN != "", "Set MAPBOX_TOKEN above"

In [ ]:
# Setup
import folium
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [ ]:
# Login to Google Cloud, for downloading images and predictions
!gcloud auth login
!gcloud config set project "bsos-geog-harvest1"

In [ ]:
# Downlaod file from Google Cloud Storage
!gcloud storage cp gs://street2sat-database-csv/street2sat-v2-points-in-field.csv /content/

In [ ]:
df_path = Path("/content/street2sat-v2-points-in-field.csv")
assert df_path.exists(), f"File not found: {df_path.name}, rerun above cell."
df = pd.read_csv(df_path)

In [ ]:
keys = ["input_img", "mapbox_img_source", "bing_img_source", "mapbox_img_segmentation", "bing_img_segmentation"]

def download_images(row: dict):
    img_root = Path(f"/content/{row['name']}")
    img_root.mkdir(exist_ok=True)
    for k in keys:
        path = img_root / f"{k}.jpg"
        if not path.exists():
            img_src = row[k]
            !gcloud storage cp {img_src} {path}
    return img_root

def plot_images(img_root: Path):
    plt.imshow(plt.imread(img_root / (keys[0] + ".jpg")))
    plt.yticks([])
    plt.xticks([])

    fig, ax = plt.subplots(2,2, figsize=(6,6))

    ax[0, 0].imshow(plt.imread(img_root / (keys[1] + ".jpg")))
    ax[1, 0].imshow(plt.imread(img_root / (keys[2] + ".jpg")))
    ax[0, 1].imshow(plt.imread(img_root / (keys[3] + ".jpg")))
    ax[1, 1].imshow(plt.imread(img_root / (keys[4] + ".jpg")))

    for i, row_title in enumerate(["Mapbox", "Bing"]):
        ax[i, 0].set_ylabel(row_title, fontsize=12)

    for j, col_title in enumerate(["Satellite Image", "Segmentation"]):
        ax[1, j].set_xlabel(col_title, fontsize=12)

    for row in ax:
        for col in row:
            col.set_yticks([])
            col.set_yticklabels([])
            col.set_xticks([])
            col.set_xticklabels([])

    plt.tight_layout()


def generate_map(row):
    m = folium.Map(
        location=eval(row["coord"]),
        zoom_start=18,
        tiles='https://api.tiles.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.png?access_token=' + MAPBOX_TOKEN,
        attr='mapbox.com'
    )

    folium.Marker(eval(row["coord"]), popup='GoPro', icon=folium.Icon(color="lightgray")).add_to(m)
    folium.Marker(eval(row["mapbox_field_coord"]), popup='Mapbox', icon=folium.Icon(color="blue")).add_to(m)
    folium.Marker(eval(row["bing_field_coord"]), popup='Bing', icon=folium.Icon(color="orange")).add_to(m)
    return m

def download_and_plot(country_folder, index):
    row = df[df["country_folder"] == country_folder].iloc[index]
    img_root = download_images(row)
    print("Copy the following line into Google sheets")
    line_part1 = f"{country_folder}\t{index}\t{row['name']}\t{row['time']}\t"
    line_part2 = f"{row['coord'][1:-1]}\t{row['dominant_crop']}"
    print(line_part1 + line_part2)
    print("")

    plot_images(img_root)


In [ ]:
# Amount of current predictions per country
df["country_folder"].value_counts()

## 3. Individual Point Analysis

1. Run the below cell specifying the desired country folder and index.

2. Run the `Visualize Imagery` cell to see the:

 a. GoPro imagery

 b. High res satellite imagery from bing and mapbox

 c. Field segmentation predictions for satellite imagery (with red centroid point)

3. Paste the printed `Road lat lon` into Google Earth to visualize all nearby points.

4. Run the `Generate Map` cell to see the coordinates of the original image coordinate from road, and predicted field coordinates from bing and mapbox.

5. Note your observations about the point in the associated Google Sheet: https://docs.google.com/spreadsheets/d/1SErjKizF6fM_48p8onQwJXWFr07rbqL2ToKh87TvDe8/edit?usp=sharing

6. Update the index in the below cell and rerun to analyze the next point.

In [ ]:
# Update the below values to analyze a specific point
country_folder = "KENYA_v2"
index = 114

# Visualize imagery
row = download_and_plot(country_folder, index)

In [ ]:
# Visualize imagery
row = download_and_plot(country_folder, index + 1)